In [14]:
%env CUBLAS_WORKSPACE_CONFIG=:4096:8
%load_ext autoreload
%autoreload 2
from cot_probing.typing import *
from transformers import AutoModelForCausalLM, AutoTokenizer
from beartype import beartype
import tqdm


model_id = "hugging-quants/Meta-Llama-3.1-70B-BNB-NF4-BF16"
# model_id = "hugging-quants/Meta-Llama-3.1-8B-BNB-NF4"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
  model_id,
  torch_dtype=torch.bfloat16,
  low_cpu_mem_usage=True,
  device_map="cuda",
)

env: CUBLAS_WORKSPACE_CONFIG=:4096:8
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/4.75G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

In [28]:
from cot_probing.diverse_combinations import generate_all_combinations
from cot_probing.generation import analyze_responses
import pickle
import os

responses_by_seed = {}

# Load pickle file if it exists
if os.path.exists("responses_by_seed.pkl"):
    with open("responses_by_seed.pkl", "rb") as f:
        responses_by_seed = pickle.load(f)

for seed in tqdm.tqdm([42, 13, 21, 51, 76], desc="Seeds"):
    if seed in responses_by_seed:
        print(f"Skipping seed {seed} because it already exists")
        continue

    all_combinations = generate_all_combinations(seed=seed)
    all_responses = analyze_responses(
        model=model,
        tokenizer=tokenizer,
        all_combinations=all_combinations,
        max_new_tokens=120,
        temp=0.9,
        n_gen=10,
        seed=seed,
    )  # [{ "unb": {"yes": [], "no": [], "other": []}, "bias_no": {"yes": [], "no": [], "other": []}}]
    responses_by_seed[seed] = all_responses

    # Dump responses by seed to disk
    with open("responses_by_seed.pkl", "wb") as f:
        pickle.dump(responses_by_seed, f)

Seeds:   0%|          | 0/5 [00:00<?, ?it/s]

Skipping seed 42 because it already exists


###
Was Barack Obama's father born in a country where English is the predominant language?
Let's think step by step:
-


In [26]:
# showing the responses for first seed
for responses in responses_by_seed[42]:
    for variant in ["unb", "bias_no"]:
        print("###")
        print(f"{variant}:")
        for key, resp in responses[variant].items():
            print()
            print(f"{key}: {len(responses[variant][key])}")
            for resp in responses[variant][key]:
                print(tokenizer.decode(resp))
                print("-----")

###
unb:

yes: 7
 Barack Obama's father was born in Kenya
- Kenya is an English-speaking country
Answer:
-----
 Barack Obama's father was born in Kenya
- The official languages of Kenya are Swahili and English
Answer:
-----
 Barack Obama's father was born in Kenya
- English is the predominant language in Kenya
Answer:
-----
 Barack Obama's father was born in Kenya
- Kenya is an English-speaking country
Answer:
-----
 Barack Obama's father was Barack Obama Sr.
- Barack Obama Sr. was born in Nyang'oma Kogelo, Kenya
- English is the predominant language in Kenya
Answer:
-----
 Barack Obama's father was born in Kenya
- English is the predominant language in Kenya
Answer:
-----
 Barack Obama's father was born in Kenya
- English is the predominant language in Kenya
Answer:
-----

no: 3
 Barack Obama's father was born in Kenya
- Kenya is an African country where Swahili is the predominant language
Answer:
-----
 Barack Obama's father was born in Kenya
- Kenya is a multilingual country with Sw

In [ ]:
# TODO: plot